<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/train_hyperdimensional_ai_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
train_hyperdimensional_ai.py

Physics‐informed AI pipeline for HyperdimensionalAI with NaN safety and uncertainty.
1. Synthetic dataset: quantum energy fluctuations, dimensional stability, entanglement, multiverse interference, hyperflux density
2. Float32 normalization and dtype consistency
3. MLP with LayerNorm, Dropout, ReLU (accepts int hidden_dims)
4. Physics‐informed residual with denominator clamps
5. MC‐Dropout for uncertainty quantification
6. Training loop: AdamW, ReduceLROnPlateau, gradient clipping, NaN checks, early stopping
7. Safe checkpoint load
8. Visualizations: loss history, scatter, uncertainty heatmap
"""

import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split

# ------------------------------------------------------------------------------
# 1. Synthetic Hyperdimensional Energy Dataset
# ------------------------------------------------------------------------------
class HyperdimensionalDataset(Dataset):
    def __init__(self, n_samples=5000, seed=42):
        np.random.seed(seed)
        # Features:
        # QEF ∈ [0.1, 1.0], DS ∈ [0.1, 10.0], EF ∈ [0.0, 1.0],
        # MI ∈ [0.0, 5.0], HD ∈ [1e2, 1e4]
        QEF = np.random.uniform(0.1,   1.0,    (n_samples,1))
        DS  = np.random.uniform(0.1,  10.0,    (n_samples,1))
        EF  = np.random.uniform(0.0,   1.0,    (n_samples,1))
        MI  = np.random.uniform(0.0,   5.0,    (n_samples,1))
        HD  = np.random.uniform(1e2, 1e4,      (n_samples,1))

        X_raw = np.hstack([QEF, DS, EF, MI, HD]).astype(np.float64)

        # Physics targets:
        # EFC (energy field coherence) = QEF * EF / (DS + eps)
        # SF  (stability factor)        = DS * MI / (EF + eps)
        # UEO (usable energy output)    = QEF * EF * HD
        eps = 1e-6
        EFC = QEF * EF / (DS + eps)
        SF  = DS  * MI / (EF + eps)
        UEO = QEF * EF * HD

        Y_raw = np.hstack([EFC, SF, UEO]).astype(np.float64)
        # add relative noise
        Y_raw += 0.02 * Y_raw.std(axis=0) * np.random.randn(*Y_raw.shape)

        # compute stats
        self.X_mean = X_raw.mean(axis=0)
        self.X_std  = X_raw.std(axis=0) + 1e-8
        self.Y_mean = Y_raw.mean(axis=0)
        self.Y_std  = Y_raw.std(axis=0) + 1e-8

        # normalize
        self.X = ((X_raw - self.X_mean) / self.X_std).astype(np.float32)
        self.Y = ((Y_raw - self.Y_mean) / self.Y_std).astype(np.float32)

        print(f"Dataset X range {self.X.min():.3e}–{self.X.max():.3e}")
        print(f"Dataset Y range {self.Y.min():.3e}–{self.Y.max():.3e}")

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return torch.from_numpy(self.X[i]), torch.from_numpy(self.Y[i])

# ------------------------------------------------------------------------------
# 2. Model Definition (accepts int hidden_dims)
# ------------------------------------------------------------------------------
class HyperdimensionalAI(nn.Module):
    def __init__(self, input_dim=5, hidden_dims=(64,64), output_dim=3, p_drop=0.1):
        super().__init__()
        # allow integer for hidden_dims
        if isinstance(hidden_dims, int):
            hidden_dims = (hidden_dims,)

        layers, dim = [], input_dim
        for h in hidden_dims:
            layers += [
                nn.Linear(dim, h),
                nn.LayerNorm(h),
                nn.ReLU(),
                nn.Dropout(p_drop)
            ]
            dim = h
        layers.append(nn.Linear(dim, output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# ------------------------------------------------------------------------------
# 3. Physics‐informed Loss with Denominator Clamps
# ------------------------------------------------------------------------------
def physics_residual(pred, X, stats):
    X_den = X * stats['X_std'] + stats['X_mean']
    QEF, DS, EF, MI, HD = X_den.t()
    eps = 1e-4

    EFC_t = QEF * EF / torch.clamp(DS + eps, min=eps)
    SF_t  = DS  * MI / torch.clamp(EF + eps, min=eps)
    UEO_t = QEF * EF * HD

    Yt = torch.stack([EFC_t, SF_t, UEO_t], dim=1)
    Yt_norm = (Yt - stats['Y_mean']) / stats['Y_std']
    return nn.MSELoss()(pred, Yt_norm)

def total_loss(pred, true, X, stats, lam=1.0):
    mse  = nn.MSELoss()(pred, true)
    phys = physics_residual(pred, X, stats)
    return mse + lam * phys, mse, phys

# ------------------------------------------------------------------------------
# 4. MC‐Dropout Uncertainty Estimation
# ------------------------------------------------------------------------------
def mc_dropout_predict(model, X, T=50):
    model.train()
    preds = []
    with torch.no_grad():
        for _ in range(T):
            preds.append(model(X))
    arr = torch.stack(preds, 0)
    return arr.mean(0), arr.std(0)

# ------------------------------------------------------------------------------
# 5. Training Loop (NaN safety, checkpoints, early stopping)
# ------------------------------------------------------------------------------
def train(model, train_loader, val_loader, stats, device,
          lr=1e-4, wd=1e-5, lam=1.0, epochs=100, patience=10):
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=5)

    best_val, wait = float('inf'), 0
    history = {'train': [], 'val': []}

    for epoch in range(1, epochs+1):
        model.train()
        train_loss = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss, _, _ = total_loss(pred, yb, xb, stats, lam)
            if torch.isnan(loss):
                print("NaN loss detected. Stopping.")
                return history

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            train_loss += loss.item() * xb.size(0)
        train_loss /= len(train_loader.dataset)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                pred = model(xb)
                loss, _, _ = total_loss(pred, yb, xb, stats, lam)
                val_loss += loss.item() * xb.size(0)
        val_loss /= len(val_loader.dataset)

        scheduler.step(val_loss)
        history['train'].append(train_loss)
        history['val'].append(val_loss)
        print(f"Epoch {epoch:03d} ┃ Train {train_loss:.4e} ┃ Val {val_loss:.4e}")

        # checkpoint & early stop
        if val_loss < best_val - 1e-8:
            best_val, wait = val_loss, 0
            torch.save(model.state_dict(), "best_hyperai.pth")
        else:
            wait += 1
            if wait >= patience:
                print("Early stopping.")
                break

    # load best
    if os.path.exists("best_hyperai.pth"):
        model.load_state_dict(torch.load("best_hyperai.pth", map_location=device))
    else:
        print("No checkpoint found; using last model.")

    return history

# ------------------------------------------------------------------------------
# 6. Visualization Helpers
# ------------------------------------------------------------------------------
def plot_history(hist):
    plt.plot(hist['train'], label='train')
    plt.plot(hist['val'],   label='val')
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

def plot_scatter(y_true, y_pred):
    plt.scatter(y_true, y_pred, s=5)
    m, M = y_true.min(), y_true.max()
    plt.plot([m, M], [m, M], 'r--')
    plt.xlabel("True")
    plt.ylabel("Pred")
    plt.show()

def plot_uncertainty(model, stats, device):
    # vary QEF (idx=0) and EF (idx=2)
    Q = np.linspace(0.1, 1.0, 80)
    E = np.linspace(0.0, 1.0, 80)
    Qm, Em = np.meshgrid(Q, E)
    pts = Qm.size

    grid = torch.zeros((pts, 5), device=device)
    grid[:, 0] = torch.from_numpy(Qm.ravel()).to(device)
    grid[:, 2] = torch.from_numpy(Em.ravel()).to(device)
    # fix DS, MI, HD at mean
    for i in (1, 3, 4):
        grid[:, i] = stats['X_mean'][i]

    Xn, _ = (grid - stats['X_mean']) / stats['X_std'], None
    _, std = mc_dropout_predict(model, Xn, T=100)
    U = std[:, 0].cpu().numpy().reshape(Qm.shape)

    plt.pcolormesh(Qm, Em, U, cmap='magma')
    plt.colorbar(label="Uncertainty σ")
    plt.xlabel("Quantum Energy Fluctuation")
    plt.ylabel("Entanglement Factor")
    plt.show()

# ------------------------------------------------------------------------------
# 7. Main Execution
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # load dataset
    dataset = HyperdimensionalDataset(n_samples=5000)
    stats = {
        'X_mean': torch.tensor(dataset.X_mean, dtype=torch.float32, device=device),
        'X_std' : torch.tensor(dataset.X_std,  dtype=torch.float32, device=device),
        'Y_mean': torch.tensor(dataset.Y_mean, dtype=torch.float32, device=device),
        'Y_std' : torch.tensor(dataset.Y_std,  dtype=torch.float32, device=device)
    }

    train_ds, val_ds = random_split(dataset, [len(dataset)-1000, 1000])
    train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
    val_loader   = DataLoader(val_ds,   batch_size=256, shuffle=False)

    # build & train model
    model = HyperdimensionalAI(input_dim=5, hidden_dims=32, output_dim=3).to(device)
    history = train(model, train_loader, val_loader, stats, device)

    # plots
    plot_history(history)
    X_all = torch.from_numpy(dataset.X).to(device)
    with torch.no_grad():
        Y_pred = model(X_all).cpu().numpy()

    for i in range(3):
        plot_scatter(dataset.Y[:, i], Y_pred[:, i])

    plot_uncertainty(model, stats, device)